In [4]:
# import cv2
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from tqdm import tqdm  # Import tqdm for progress bar

# # Define function: Convert to Ohta color space
# def convert_to_ohta(image_rgb):
#     R, G, B = image_rgb[:,:,0], image_rgb[:,:,1], image_rgb[:,:,2]
#     I1 = (R + G + B) / 3.0  # Brightness component
#     I2 = (R - B) / 2.0      # Red-blue difference
#     I3 = (2 * G - R - B) / 4.0  # Green-purple difference
#     return I1, I2, I3

# # Define function: Convert to LST color space
# def convert_to_lst(image_rgb):
#     R, G, B = image_rgb[:,:,0], image_rgb[:,:,1], image_rgb[:,:,2]
#     L = (R + G + B) / 3.0  # Brightness
#     S = np.sqrt(((R - G)**2 + (R - B)**2 + (G - B)**2) / 3.0)  # Saturation
#     T = (R - B)  # Hue
#     return L, S, T

# # Read the CSV file
# data_path = '/home/ubuntu/landscape-aesthetics/data/external/scenicornot/scenicornot.metadata.csv'
# image_folder = Path('/home/ubuntu/landscape-aesthetics/data/external/scenicornot')

# data = pd.read_csv(data_path)

# # List to store the final results
# results = []

# # Iterate over each row in the CSV file with tqdm progress bar
# for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing images"):
#     # Read image path
#     image_path = image_folder / row['filename']  # Assuming 'filename' is the column name containing image file names
    
#     try:
#         # Read the image
#         image = cv2.imread(str(image_path))
        
#         if image is None:
#             print(f"Image not found: {image_path}, skipping...")
#             continue
        
#         # Convert BGR to RGB (OpenCV reads in BGR format by default)
#         image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         # Convert to HSV color space
#         image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#         H, S, V = image_hsv[:,:,0], image_hsv[:,:,1], image_hsv[:,:,2]
        
#         # Convert to Ohta color space
#         I1, I2, I3 = convert_to_ohta(image_rgb)
        
#         # Convert to LST color space
#         L, S_lst, T = convert_to_lst(image_rgb)
        
#         # Append the average values of each color space to the results list
#         results.append({
#             'image_path': str(image_path),
#             'H_mean': np.mean(H), 'S_mean': np.mean(S), 'V_mean': np.mean(V),
#             'I1_mean': np.mean(I1), 'I2_mean': np.mean(I2), 'I3_mean': np.mean(I3),
#             'L_mean': np.mean(L), 'S_lst_mean': np.mean(S_lst), 'T_mean': np.mean(T)
#         })
    
#     except Exception as e:
#         print(f"Error processing image {image_path}: {e}")
#         continue  # Skip this image and move to the next one

# # Convert results to a DataFrame
# output_df = pd.DataFrame(results)

# # Save results to a new CSV file
# output_df.to_csv('/home/ubuntu/landscape-aesthetics/data/external/scenicornot/scenicornot_color_feature.csv', index=False)

# print("Color space extraction is complete, results saved in 'scenicornot_color_feature.csv'.")


Processing images:   9%|█▎             | 18604/211856 [09:48<1:41:57, 31.59it/s]


KeyboardInterrupt: 

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from PIL import Image
from pathlib import Path
from torchvision import transforms
from tqdm import tqdm  # Import tqdm for progress bar

# Set paths
data_path = '/home/ubuntu/landscape-aesthetics/data/external/scenicornot/scenicornot.metadata.csv'
image_folder = Path('/home/ubuntu/landscape-aesthetics/data/external/scenicornot') 
data = pd.read_csv(data_path)

# Define image transformations
data_transforms = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

# List to store image paths and dominant colors
dominant_colors_data = []

# Loop through each image with a progress bar
for idx in tqdm(range(len(data)), desc="Processing Images"):
    img_name = data.iloc[idx]['filename']
    image_path = image_folder / Path(img_name)
    try:
        # Read and process the image
        image = Image.open(image_path).convert('RGB')
        if image is None:
            print(f"Warning: Unable to load image {image_path}, skipping...")
            continue  # Skip if the image cannot be loaded

        # Apply transformations
        img = data_transforms(image)
        
        # Extract dominant colors
        image_np = np.array(image)
        image_np = image_np.reshape(-1, 3)  # Reshape image data to 2D array
        kmeans = KMeans(n_clusters=3, random_state=0).fit(image_np)
        dominant_color = kmeans.cluster_centers_  # Get top three dominant colors
        
        # Add image path and dominant colors to results list
        dominant_colors_data.append([str(image_path), *dominant_color.flatten()])
        
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        continue  # Skip this image and proceed if an error occurs

# Save results to CSV file
output_df = pd.DataFrame(dominant_colors_data, columns=['image_path', 'R1', 'G1', 'B1', 'R2', 'G2', 'B2', 'R3', 'G3', 'B3'])
output_df.to_csv('/home/ubuntu/landscape-aesthetics/reports/dominant_colors.csv', index=False)


Processing Images:   0%|                | 212/211856 [00:44<12:17:57,  4.78it/s]


KeyboardInterrupt: 